In [3]:
from datetime import datetime, timezone


def date_to_utc_timestamp(c_date):
    return int(c_date.replace(tzinfo=timezone.utc).timestamp())


def utc_timestamp_to_date(ts):
    return datetime.fromtimestamp(ts, tz=timezone.utc)


start_date = datetime(2023, 5, 1)
print(date_to_utc_timestamp(start_date))
# print(utc_timestamp_to_date(1626480000))

1682899200


In [11]:
# 1. Fetch Eth Pool Liquidity Data

from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport

# 0.05% WBTC/ETH pool
LOW_POOL_ID = "0x4585fe77225b41b697c938b018e2ac67ac5a20c0"
# 0.3% WBTC/ETH pool
HIGH_POOL_ID = "0xcbcdf9626bc03e24f779434178a73a0b4bad62ed"


def get_pool_trade_data(pool_id, start_date):
    pool_query = """
    query poolDayDatas($pool_id: ID!, $start_time: Int!,){
        poolDayDatas(
            where: { pool: $pool_id, date_gt: $start_time }
        orderBy: date
        orderDirection: asc
        ) {
            date
            # in range liquidity at end of period
            liquidity
            # current price tracker at end of period
            sqrtPrice
            # price of token0 - derived from sqrtPrice
            tick
            # tvl derived in USD at end of period
            tvlUSD
            # volume in token0
            volumeToken0
            # volume in token1
            volumeToken1
            # volume in USD
            volumeUSD
            # fees in USD
            feesUSD
            # number of transactions during period
            txCount
            # opening price of token0
            open
            # high price of token0
            high
            # low price of token0
            low
            # close price of token0
            close
            }
    }
    """

    client = Client(
        transport=RequestsHTTPTransport(
            url='https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3',
            verify=True,
            retries=5,
        ))

    start_timestamp = date_to_utc_timestamp(start_date)
    variables = {"pool_id": pool_id, "start_time": start_timestamp}
    response = client.execute(gql(pool_query), variable_values=variables)
    trade_data = response["poolDayDatas"]
    trade_all_data = trade_data

    while len(trade_data) > 0:
        print(utc_timestamp_to_date(start_timestamp))
        print(len(trade_all_data))
        start_timestamp = trade_data[-1]['date']
        variables = {"pool_id": pool_id, "start_time": start_timestamp}
        response = client.execute(gql(pool_query), variable_values=variables)
        trade_data = response["poolDayDatas"]
        trade_all_data += trade_data

    return trade_all_data


# the earliest data is from 2021-05-01
start_date = datetime(2021, 5, 1)
pool_data = get_pool_trade_data(LOW_POOL_ID, start_date)

2021-05-01 00:00:00+00:00
100
2021-08-12 00:00:00+00:00
200
2021-11-20 00:00:00+00:00
300
2022-02-28 00:00:00+00:00
400
2022-06-08 00:00:00+00:00
500
2022-09-16 00:00:00+00:00
600
2022-12-25 00:00:00+00:00
700
2023-04-04 00:00:00+00:00
732


In [4]:
import pandas as pd

uniswap_df = pd.DataFrame(pool_data)
uniswap_df.head()

,date,liquidity,sqrtPrice,tick,tvlUSD,volumeToken0,volumeToken1,volumeUSD,feesUSD,txCount,open,high,low,close
0,1620172800,0,0,None,0,0,0,0,0,2,0,0,0,0
1,1620259200,47802696926688,31809195401932648817609369003249547,258071,5146.736482364860299518079151516243,0.01955837,0.316827345146869434,1102.031018311199696439388175003343,0.5510155091555998482196940875016715,16,0.05998654715076262372902751818670286,0.06243654451824971693450264979198528,0.05998654715076262372902751818670286,0.05998654715076262372902751818670286
2,1620345600,42425055535734978,32187992991029178621379247853387443,258308,2073400.162975270801081924615102077,68.18328357,1120.626959335943537703,3922263.61944071584148036405495539,1961.131809720357920740182027477691,227,0.06203743019377815473808300194149325,0.06203743019377815473808300194149325,0.06010084251900176152287862020188552,0.06203743019377815473808300194149325
3,1620432000,4036037627994723,30814946235335421635470353514297570,257436,1980521.049385023017241543520869319,126.96928506,2072.7905681432721063,7443618.326347876067605645380688426,3721.809163173938033802822690344206,257,0.06058403976932998513307639824064359,0.06669932160003833798769060727416865,0.06004384605589742079211220851145031,0.06058403976932998513307639824064359
4,1620518400,672440555897057,30550018995856811291234861690516572,257263,739735.1727825094073093447014429385,10.91169152,163.331111671204485144,634883.3690065130877645983992275799,317.4416845032565438822991996137902,99,0.06609184203913069571728478548803438,0.06830342385985722044198599944892239,0.0653848260519338954289133091895779,0.06609184203913069571728478548803438


In [5]:
uniswap_df.to_csv("../data/wbtc_eth_0.05_uniswap_data.csv", index=False)

In [10]:
pool_data1 = get_pool_trade_data(HIGH_POOL_ID, start_date)
uniswap_df1 = pd.DataFrame(pool_data1)
uniswap_df1.to_csv("../data/wbtc_eth_0.3_uniswap_data.csv", index=False)

2021-05-01 00:00:00+00:00
100
2021-08-11 00:00:00+00:00
200
2021-11-19 00:00:00+00:00
300
2022-02-27 00:00:00+00:00
400
2022-06-07 00:00:00+00:00
500
2022-09-15 00:00:00+00:00
600
2022-12-24 00:00:00+00:00
700
2023-04-03 00:00:00+00:00
733


In [1]:
# 2. Fetch Arb Swap Period Data

from datetime import datetime
from utils import utils


start_timestamp = utils.date_to_utc_timestamp(datetime(2023, 5, 28, 0, 0))
end_timestamp = utils.date_to_utc_timestamp(datetime(2023, 5, 30, 0, 0))

In [2]:
import pandas as pd

arb_uniswap_df = pd.read_csv("../data/arb_swap_0523_0528.csv")
id_list = arb_uniswap_df["id"].tolist()
print(id_list[0])

0x13875c5e00916a733dbf870b03f29c7c0e2b575294ea3c17e3a123540b281eca0b000000


In [3]:
# ARB graphQL
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport


arb_query = """
query {
    swaps(skip: %d, orderBy: timestamp, orderDirection: asc,
        where: { pool: "0xc31e54c7a869b9fcbecc14363cf510d1c41fa443", timestamp_gte: %d }){
            id
            account {
                id
                positionCount
                openPositionCount
                closedPositionCount
                depositCount
                withdrawCount
                swapCount
            }
            gasLimit
            gasPrice
            amountIn
            amountInUSD
            amountOut
            amountOutUSD
            tick
            timestamp
            tokenIn {
                symbol
            }
            tokenOut {
                symbol
            }
    }
}
"""


client = Client(
    transport=RequestsHTTPTransport(
        url='https://api.thegraph.com/subgraphs/name/messari/uniswap-v3-arbitrum',
        verify=True,
        retries=5,
    ))

# # variables = { "num_skip": 0, "start_timestamp": 1684460940 }
# # response = client.execute(gql(arb_query), variable_values=variables)
response = client.execute(gql(arb_query % (0, start_timestamp)))
print(len(response["swaps"]))

100


In [4]:
result = response['swaps']
id_set = set(id_list)
print_set = set()
PRINT_GAP = 1000

for item in result:
    id_set.add(item["id"])

# more than 100 swap transactions happened in a second, will need to change skip count
skip = 0
last_cnt = 0
while int(result[-1]["timestamp"]) <= end_timestamp:
    if skip == 1000:
        break
    last_cnt = len(result)

    print_cnt = int(len(result) / PRINT_GAP)
    if print_cnt not in print_set:
        print_set.add(print_cnt)
        print(datetime.now(), "result cnt:", len(result), "id cnt:", len(id_set), "skip: ", skip)
        print("Current time: ", utils.utc_timestamp_to_date(int(result[-1]["timestamp"])))

    current_timestamp = int(result[-1]["timestamp"])
    query = arb_query % (skip, current_timestamp)
    try:
        response = client.execute(gql(query))

        swap_data = response["swaps"]
        if len(swap_data) == 0:
            break
        for item in swap_data:
            if int(item["timestamp"]) > end_timestamp:
                break
            elif item["id"] in id_set:
                continue
            else:
                result.append(item)
                id_set.add(item["id"])

        if last_cnt == len(result):
            skip += 100
        else:
            skip = 0
    except Exception as ex:
        print("Fetching swap data error", ex)

2023-05-30 09:38:53.015469 result cnt: 100 id cnt: 68938 skip:  0
Current time:  2023-05-28 00:16:27+00:00
2023-05-30 09:39:37.399397 result cnt: 1057 id cnt: 69895 skip:  0
Current time:  2023-05-28 00:58:53+00:00
2023-05-30 09:40:23.179774 result cnt: 2045 id cnt: 70883 skip:  0
Current time:  2023-05-28 03:10:10+00:00
2023-05-30 09:41:09.920128 result cnt: 3028 id cnt: 71866 skip:  0
Current time:  2023-05-28 05:09:12+00:00
2023-05-30 09:41:55.994083 result cnt: 4011 id cnt: 72849 skip:  0
Current time:  2023-05-28 08:22:33+00:00
2023-05-30 09:42:45.515943 result cnt: 5098 id cnt: 73936 skip:  0
Current time:  2023-05-28 10:52:32+00:00
2023-05-30 09:43:31.016815 result cnt: 6082 id cnt: 74920 skip:  0
Current time:  2023-05-28 12:46:04+00:00
2023-05-30 09:44:15.412316 result cnt: 7069 id cnt: 75907 skip:  0
Current time:  2023-05-28 15:05:32+00:00
2023-05-30 09:44:59.191941 result cnt: 8056 id cnt: 76894 skip:  0
Current time:  2023-05-28 17:45:30+00:00
2023-05-30 09:45:47.355301 re

In [5]:
import pandas as pd
from copy import deepcopy


arb_result = deepcopy(result)
for item in arb_result:
    item["transaction"] = item["id"]
    item["trader"] = item["account"]["id"]
    item["traderPositionCount"] = item["account"]["positionCount"]
    item["traderOpenPositionCount"] = item["account"]["openPositionCount"]
    item["traderClosedPositionCount"] = item["account"]["closedPositionCount"]
    item["traderDepositCount"] = item["account"]["depositCount"]
    item["traderWithdrawCount"] = item["account"]["withdrawCount"]
    item["traderSwapCount"] = item["account"]["swapCount"]
    item["tokenIn"] = item["tokenIn"]["symbol"]
    item["tokenOut"] = item["tokenOut"]["symbol"]
    item.pop("account")

arb_uniswap_df1 = pd.DataFrame(arb_result)
df = pd.concat([arb_uniswap_df, arb_uniswap_df1])
df.head()

,id,gasLimit,gasPrice,amountIn,amountInUSD,amountOut,amountOutUSD,tick,timestamp,tokenIn,tokenOut,transaction,trader,traderPositionCount,traderOpenPositionCount,traderClosedPositionCount,traderDepositCount,traderWithdrawCount,traderSwapCount
0,0x13875c5e00916a733dbf870b03f29c7c0e2b575294ea...,8000000,400000000,18509968,18.509968,10174988585088354,18.500713,-201264,1684800030,USDC,WETH,0x13875c5e00916a733dbf870b03f29c7c0e2b575294ea...,0x112bf8e7431bde9bd9395d44307f788a6e054691,0,0,0,3695,3695,3695
1,0xa629bdac51ec7b793a04193f87006f0ff4ae910f8025...,28410397,200000000,417246407,417.246407,229361579040734245,417.037599,-201264,1684800044,USDC,WETH,0xa629bdac51ec7b793a04193f87006f0ff4ae910f8025...,0x59c5c9045374693b384bc023fdca4685c7ec1a9e,0,0,0,5046,5046,5046
2,0x38d8de741acaf8f47d3e1692926b7a663e93b6fc0e4b...,2589363,100000000,15034268999359802,27.336143,27322474,27.322474,-201264,1684800056,WETH,USDC,0x38d8de741acaf8f47d3e1692926b7a663e93b6fc0e4b...,0x679d1c9f327257c4cec9ebc1c7c750728351e989,0,0,0,410,410,410
3,0x452dd1fbd5a7c6d03880cd92be8bc35646422b627c34...,6430718,100000000,40248724,40.248724,22124832513497315,40.228597,-201264,1684800071,USDC,WETH,0x452dd1fbd5a7c6d03880cd92be8bc35646422b627c34...,0x2b185c762f6fb49a07116ec06b238b58fe2bec53,3,3,0,293,297,290
4,0x7ffaad3f6e94d4e7d19df8b5f8c808512918acccf5f1...,2526058,150000000,54000000000000000,98.185802,98136699,98.136699,-201264,1684800077,WETH,USDC,0x7ffaad3f6e94d4e7d19df8b5f8c808512918acccf5f1...,0x23e08e6e721a0a56989c8759afa1841cd07e9565,0,0,0,4,4,4


In [11]:
def get_amount_price(swap_item):
    if swap_item["tokenIn"] == "WETH":
        eth_amount = int(swap_item["amountIn"]) / (10 ** 18)
        usd = float(swap_item["amountInUSD"])
    else:
        eth_amount = int(swap_item["amountOut"]) / (10 ** 18)
        usd = float(swap_item["amountOutUSD"])

    if eth_amount != 0:
        return eth_amount, usd, usd / eth_amount

    return eth_amount, usd, None

In [17]:
data = df.to_dict("records")

for item in data:
    eth_amount, usd_amount, price = get_amount_price(item)
    item["ethAmount"] = eth_amount
    item["usdAmount"] = usd_amount
    item["price"] = price
    item.pop("id")

df_new = pd.DataFrame(data)
df_new.head()

,gasLimit,gasPrice,amountIn,amountInUSD,amountOut,amountOutUSD,tick,timestamp,tokenIn,tokenOut,...,trader,traderPositionCount,traderOpenPositionCount,traderClosedPositionCount,traderDepositCount,traderWithdrawCount,traderSwapCount,ethAmount,usdAmount,price
0,8000000,400000000,18509968,18.509968,10174988585088354,18.500713,-201264,1684800030,USDC,WETH,...,0x112bf8e7431bde9bd9395d44307f788a6e054691,0,0,0,3695,3695,3695,0.010175,18.500713,1818.25389
1,28410397,200000000,417246407,417.246407,229361579040734245,417.037599,-201264,1684800044,USDC,WETH,...,0x59c5c9045374693b384bc023fdca4685c7ec1a9e,0,0,0,5046,5046,5046,0.229362,417.037599,1818.25396
2,2589363,100000000,15034268999359802,27.336143,27322474,27.322474,-201264,1684800056,WETH,USDC,...,0x679d1c9f327257c4cec9ebc1c7c750728351e989,0,0,0,410,410,410,0.015034,27.336143,1818.25555
3,6430718,100000000,40248724,40.248724,22124832513497315,40.228597,-201264,1684800071,USDC,WETH,...,0x2b185c762f6fb49a07116ec06b238b58fe2bec53,3,3,0,293,297,290,0.022125,40.228597,1818.25545
4,2526058,150000000,54000000000000000,98.185802,98136699,98.136699,-201264,1684800077,WETH,USDC,...,0x23e08e6e721a0a56989c8759afa1841cd07e9565,0,0,0,4,4,4,0.054000,98.185802,1818.25560


In [25]:
df_new.to_csv("../data/arb_swap.csv", index=False)

In [4]:
# 3. Fetch Arb User Swap Data
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport

swap_query = """
{
    swaps(skip: %d, orderBy: timestamp, orderDirection: asc,
        where: { account_: {id: "0x8cc02c2381b7c55e18dccfea917f0677a5671931"}, timestamp_gte: %d }){
        id
        amountIn
        amountInUSD
        amountOut
        amountOutUSD
        gasLimit
        gasPrice
        tick
        timestamp
        pool {
            id
        }
        tokenIn {
            symbol
        }
        tokenOut {
            symbol
        }
    }
}"""


client = Client(
    transport=RequestsHTTPTransport(
        url='https://api.thegraph.com/subgraphs/name/messari/uniswap-v3-arbitrum',
        verify=True,
        retries=5,
    ))

response = client.execute(gql(swap_query % (0, 0)))
print(response["swaps"])

[{'id': '0x4163f1437cefc0b6c73e00c8db819ea92aa07395a1a666250da730c3cb8b7ade03000000', 'amountIn': '3516117741730628903', 'amountInUSD': '5174.51464616665305507211904', 'amountOut': '5170663639', 'amountOutUSD': '5170.663639', 'gasLimit': '3000000', 'gasPrice': '2000000000', 'tick': '-203382', 'timestamp': '1676306229', 'pool': {'id': '0xc31e54c7a869b9fcbecc14363cf510d1c41fa443'}, 'tokenIn': {'symbol': 'WETH'}, 'tokenOut': {'symbol': 'USDC'}}, {'id': '0x058d12a79ee20e82379e5d0b2b1eeeaaa2988768244d1ff805265603eb96d5b70e000000', 'amountIn': '4186892251423356044', 'amountInUSD': '6158.12730481500526913236532', 'amountOut': '6154400504', 'amountOutUSD': '6154.400504', 'gasLimit': '3000000', 'gasPrice': '2000000000', 'tick': '-203387', 'timestamp': '1676306236', 'pool': {'id': '0xc31e54c7a869b9fcbecc14363cf510d1c41fa443'}, 'tokenIn': {'symbol': 'WETH'}, 'tokenOut': {'symbol': 'USDC'}}, {'id': '0xcd8af3e9679b087671fa11627d3f0864b3bedb46523c158dfca748ed5c62c48d04000000', 'amountIn': '355944140

In [7]:
from utils import utils
from datetime import datetime

result = response['swaps']
id_set = set([t["id"] for t in result])
print_set = set()
PRINT_GAP = 1000

current_timestamp = int(result[-1]["timestamp"])
end_timestamp = utils.date_to_utc_timestamp(datetime(2023, 6, 1, 0, 0))

# more than 100 swap transactions happened in a second, will need to change skip count
skip = 0
last_cnt = 0
while current_timestamp <= end_timestamp:
    if skip == 1000:
        break

    print_cnt = int(len(result) / PRINT_GAP)
    if print_cnt not in print_set:
        print_set.add(print_cnt)
        print(datetime.now(), "result cnt:", len(result), "id cnt:", len(id_set), "skip: ", skip)
        print("Current time: ", utils.utc_timestamp_to_date(current_timestamp))

    last_cnt = len(result)
    query = swap_query % (skip, current_timestamp)
    try:
        response = client.execute(gql(query))

        swap_data = response["swaps"]
        if len(swap_data) == 0:
            break
        for item in swap_data:
            if int(item["timestamp"]) > end_timestamp:
                break
            elif item["id"] in id_set:
                continue
            else:
                result.append(item)
                id_set.add(item["id"])

        if last_cnt == len(result):
            skip += 100
        else:
            skip = 0

        current_timestamp = int(result[-1]["timestamp"])
    except Exception as ex:
        print("Fetching swap data error", ex)

2023-06-01 20:30:35.504524 result cnt: 100 id cnt: 100 skip:  0
Current time:  2023-02-13 17:28:41+00:00
2023-06-01 20:31:29.045159 result cnt: 1090 id cnt: 1090 skip:  0
Current time:  2023-02-14 17:21:46+00:00
2023-06-01 20:32:16.711624 result cnt: 2080 id cnt: 2080 skip:  0
Current time:  2023-02-15 19:47:13+00:00
2023-06-01 20:33:04.625562 result cnt: 3070 id cnt: 3070 skip:  0
Current time:  2023-02-16 14:42:14+00:00
2023-06-01 20:33:53.602710 result cnt: 4060 id cnt: 4060 skip:  0
Current time:  2023-02-17 00:01:10+00:00
2023-06-01 20:34:42.618796 result cnt: 5050 id cnt: 5050 skip:  0
Current time:  2023-02-17 21:10:55+00:00
2023-06-01 20:35:31.143546 result cnt: 6040 id cnt: 6040 skip:  0
Current time:  2023-02-19 17:07:57+00:00
2023-06-01 20:36:18.833062 result cnt: 7030 id cnt: 7030 skip:  0
Current time:  2023-02-20 23:02:11+00:00
2023-06-01 20:37:05.405891 result cnt: 8020 id cnt: 8020 skip:  0
Current time:  2023-02-21 23:02:09+00:00
2023-06-01 20:37:51.707454 result cnt: 

In [9]:
print(result[0])

{'id': '0x4163f1437cefc0b6c73e00c8db819ea92aa07395a1a666250da730c3cb8b7ade03000000', 'amountIn': '3516117741730628903', 'amountInUSD': '5174.51464616665305507211904', 'amountOut': '5170663639', 'amountOutUSD': '5170.663639', 'gasLimit': '3000000', 'gasPrice': '2000000000', 'tick': '-203382', 'timestamp': '1676306229', 'pool': {'id': '0xc31e54c7a869b9fcbecc14363cf510d1c41fa443'}, 'tokenIn': {'symbol': 'WETH'}, 'tokenOut': {'symbol': 'USDC'}}


In [13]:
import pandas as pd
from copy import deepcopy


arb_result = deepcopy(result)
for item in arb_result:
    item["tokenIn"] = item["tokenIn"]["symbol"]
    item["tokenOut"] = item["tokenOut"]["symbol"]
    item["pool"] = item["pool"]["id"]

arb_uniswap_df = pd.DataFrame(arb_result)
arb_uniswap_df.to_csv("../data/user_arb_swap.csv", index=False)